In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import pandas as pd
import numpy as np
import json

In [22]:
#--------------------------------------------------------------------------------------------------------------------------
#    New interaction data (annotated-stories.json)
#--------------------------------------------------------------------------------------------------------------------------

In [23]:
annotated_stories_df = pd.read_json('data/new-annotated-stories.json')
annotated_stories_df

,_datasetid,_id,_timestamp,_timestamp_day,_timestamp_hour,_timestamp_minute,_timestamp_month,_timestamp_second,_timestamp_year,activityId,authorUsername,parts,sessionName,title,_updated
0,816c8fb3-6c73-4405-bf71-b1c765c4a79e,633869df947d991ed04cf7f7,1664641503,1,17,25,10,3,2022,62cc4f819f7c9c7e6739a46b,RQ4a2nIG,"[{'artworkId': '39416', 'emotions': {'emotion:...",notte,Gli scacchi.,NaN
1,816c8fb3-6c73-4405-bf71-b1c765c4a79e,633865fd947d991ed04cf7de,1664640509,1,17,8,10,29,2022,62cc4f819f7c9c7e6739a46b,RQ4a2nIG,"[{'artworkId': '39426', 'emotions': {'emotion:...",notte,Vita,NaN
2,816c8fb3-6c73-4405-bf71-b1c765c4a79e,633865ce947d991ed04cf7dc,1664640462,1,17,7,10,42,2022,62cc4f819f7c9c7e6739a46b,8sAIqaGe,"[{'artworkId': '35433', 'emotions': {}, 'multi...",notte,Casa,NaN
3,816c8fb3-6c73-4405-bf71-b1c765c4a79e,6338649a2df2fd104d686349,1664640154,1,17,2,10,34,2022,62cc4f819f7c9c7e6739a46b,RQ4a2nIG,"[{'artworkId': '35227', 'emotions': {}, 'multi...",notte,Ritorno alle origini,NaN
4,816c8fb3-6c73-4405-bf71-b1c765c4a79e,6338625f758e693b2f1aea9d,1664639583,1,16,53,10,3,2022,62cc4f819f7c9c7e6739a46b,Sz5Wx2XD,"[{'artworkId': '35571', 'emotions': {}, 'multi...",notte,Forza potenziale,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,816c8fb3-6c73-4405-bf71-b1c765c4a79e,62272bca8b5d531b7a287bc6,1656345373,27,16,56,6,13,2022,620e37ba17f14811fa3ff880,anonymous,"[{'artworkId': '35229', 'emotions': {'emotion:...",FakeSessionName,Storia di Rossana,1.0
96,816c8fb3-6c73-4405-bf71-b1c765c4a79e,62272dc4aa89621f0d6ce920,1656345373,27,16,56,6,13,2022,620e37ba17f14811fa3ff880,anonymous,"[{'artworkId': '35400', 'emotions': {}, 'multi...",FakeSessionName,ttttt,1.0
97,816c8fb3-6c73-4405-bf71-b1c765c4a79e,6227300d925cbb70f01278b4,1656345373,27,16,56,6,13,2022,620e37ba17f14811fa3ff880,anonymous,"[{'artworkId': '41214', 'emotions': {'emotion:...",FakeSessionName,Gggg,1.0
98,816c8fb3-6c73-4405-bf71-b1c765c4a79e,622731a552655f48ae32ca9d,1656345373,27,16,56,6,13,2022,620e37ba17f14811fa3ff880,anonymous,"[{'artworkId': '35229', 'emotions': {'emotion:...",FakeSessionName,fewfew,1.0


In [24]:
# Prepare interaction data for clustering
df = annotated_stories_df.explode('parts')
df = df.reset_index()

df2 = pd.json_normalize(df["parts"], max_level = 0)
df3 = pd.concat([df,df2], axis=1, join='inner')

# Set default values
"""
values = {'emotions':  {}}
df3 = df3.fillna(value=values)
"""
df3['emotions'] = df3['emotions'].apply(lambda x: {} if x != x else x)

df4 = df3.groupby("authorUsername").agg(list)
df4 = df4.reset_index()
df5 = df4.rename(columns = {'authorUsername':'userName'})

# Combine with user data
users = pd.read_json('data/GAMGame_users_RN_UNITO.json')
user_interactions = pd.merge(df5, users, on='userName', how='left')

# Set default values
values = {'relationship_with_arts': '', 'relationship_with_museums': ''}
user_interactions = user_interactions.fillna(value=values)

user_interactions

,userName,index,_datasetid_x,_id_x,_timestamp_x,_timestamp_day_x,_timestamp_hour_x,_timestamp_minute_x,_timestamp_month_x,_timestamp_second_x,...,content_breaking,_datasetid_y,_timestamp_y,_timestamp_year_y,_timestamp_month_y,_timestamp_day_y,_timestamp_hour_y,_timestamp_minute_y,_timestamp_second_y,_updated_y
0,0bAAwK14,"[73, 73, 74, 74, 75, 75, 75, 76, 76, 76]","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[63313ffc5484a21e957f308c, 63313ffc5484a21e957...","[1664172028, 1664172028, 1664171640, 166417164...","[26, 26, 26, 26, 26, 26, 26, 23, 23, 23]","[7, 7, 6, 6, 6, 6, 6, 14, 14, 14]","[0, 0, 54, 54, 48, 48, 48, 16, 16, 16]","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9]","[28, 28, 0, 0, 54, 54, 54, 47, 47, 47]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,61f29227b9936b5c047f362a,"[91, 91, 92, 92, 93, 93, 93]","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[620e38237b1b9f66c20fe425, 620e38237b1b9f66c20...","[1656345373, 1656345373, 1656345373, 165634537...","[27, 27, 27, 27, 27, 27, 27]","[16, 16, 16, 16, 16, 16, 16]","[56, 56, 56, 56, 56, 56, 56]","[6, 6, 6, 6, 6, 6, 6]","[13, 13, 13, 13, 13, 13, 13]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6rK3p7za,"[41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 4...","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[6333fd5713a6b278b06dc31b, 6333fd5713a6b278b06...","[1664351575, 1664351575, 1664351575, 166435133...","[28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 2...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[52, 52, 52, 48, 48, 48, 44, 44, 44, 39, 39, 3...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[55, 55, 55, 59, 59, 59, 39, 39, 39, 0, 0, 0, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8sAIqaGe,"[2, 2]","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[633865ce947d991ed04cf7dc, 633865ce947d991ed04...","[1664640462, 1664640462]","[1, 1]","[17, 17]","[7, 7]","[10, 10]","[42, 42]",...,Non mi interessano i contenuti in Lingua dei S...,c463550f-12ca-47d8-a1f7-8fda690781cb,1.665122e+09,2022.0,10.0,7.0,7.0,0.0,50.0,True
4,B3N0HGw4,"[11, 11, 11]","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[633858f82b631c1bf17add1d, 633858f82b631c1bf17...","[1664637177, 1664637177, 1664637177]","[1, 1, 1]","[16, 16, 16]","[12, 12, 12]","[10, 10, 10]","[57, 57, 57]",...,Vorrei vedere i contenuti in Lingua dei Segni ...,c463550f-12ca-47d8-a1f7-8fda690781cb,1.665122e+09,2022.0,10.0,7.0,7.0,0.0,54.0,True
5,BTKF72et,"[69, 69, 69, 77, 77, 77, 78, 78, 78]","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[6331680d71f6471dbc6d99d6, 6331680d71f6471dbc6...","[1664182285, 1664182285, 1664182285, 166393830...","[26, 26, 26, 23, 23, 23, 23, 23, 23]","[9, 9, 9, 14, 14, 14, 13, 13, 13]","[51, 51, 51, 5, 5, 5, 49, 49, 49]","[9, 9, 9, 9, 9, 9, 9, 9, 9]","[25, 25, 25, 7, 7, 7, 3, 3, 3]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,FVhf7OMw,"[9, 9, 9]","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[63385bb4be73e50c31468039, 63385bb4be73e50c314...","[1664637876, 1664637876, 1664637876]","[1, 1, 1]","[16, 16, 16]","[24, 24, 24]","[10, 10, 10]","[36, 36, 36]",...,Non mi interessano i contenuti in Lingua dei S...,c463550f-12ca-47d8-a1f7-8fda690781cb,1.665122e+09,2022.0,10.0,7.0,7.0,1.0,0.0,True
7,KQTGApjD,"[13, 13, 13]","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[633857981931f9059341ba94, 633857981931f905934...","[1664636824, 1664636824, 1664636824]","[1, 1, 1]","[16, 16, 16]","[7, 7, 7]","[10, 10, 10]","[4, 4, 4]",...,Non mi interessano i contenuti in Lingua dei S...,c463550f-12ca-47d8-a1f7-8fda690781cb,1.665122e+09,2022.0,10.0,7.0,7.0,0.0,59.0,True
8,L11lvHun,"[79, 79]","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[632d5c10719af0778157a991, 632d5c10719af077815...","[1663917072, 1663917072]","[23, 23]","[8, 8]","[11, 11]","[9, 9]","[12, 12]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LW3Klzvx,"[10, 10, 10, 10, 10]","[816c8fb3-6c73-4405-bf71-b1c765c4a79e, 816c8fb...","[6338593d2b631c1bf17add24, 6338593d2b631c1bf17...","[1664637245, 1664637245, 16646372

In [30]:
result = user_interactions.to_json(orient='records')
parsed = json.loads(result)
parsed

[{'userName': '0bAAwK14',
  'index': [73, 73, 74, 74, 75, 75, 75, 76, 76, 76],
  '_datasetid_x': ['816c8fb3-6c73-4405-bf71-b1c765c4a79e',
   '816c8fb3-6c73-4405-bf71-b1c765c4a79e',
   '816c8fb3-6c73-4405-bf71-b1c765c4a79e',
   '816c8fb3-6c73-4405-bf71-b1c765c4a79e',
   '816c8fb3-6c73-4405-bf71-b1c765c4a79e',
   '816c8fb3-6c73-4405-bf71-b1c765c4a79e',
   '816c8fb3-6c73-4405-bf71-b1c765c4a79e',
   '816c8fb3-6c73-4405-bf71-b1c765c4a79e',
   '816c8fb3-6c73-4405-bf71-b1c765c4a79e',
   '816c8fb3-6c73-4405-bf71-b1c765c4a79e'],
  '_id_x': ['63313ffc5484a21e957f308c',
   '63313ffc5484a21e957f308c',
   '63313e7848ebba445a3b3cee',
   '63313e7848ebba445a3b3cee',
   '63313d4648ebba445a3b3ce9',
   '63313d4648ebba445a3b3ce9',
   '63313d4648ebba445a3b3ce9',
   '632db1bfcd877313115bb376',
   '632db1bfcd877313115bb376',
   '632db1bfcd877313115bb376'],
  '_timestamp_x': [1664172028,
   1664172028,
   1664171640,
   1664171640,
   1664171334,
   1664171334,
   1664171334,
   1663939007,
   1663939007,
   

In [32]:
with open("data/generated/annotated_stories_merged.json", "w") as outfile:
    #json.dump(user_interactions.to_dict('records'), outfile, indent=4)
    json.dump(parsed, outfile, indent=4)